In [1]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training

import os
import torch
import torch.nn as nn

c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll
CUDA SETUP: CUDA runtime path found: C:\Users\panta\anaconda3\envs\nlp\bin\cudart64_110.dll
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll...


c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: C:\Users\panta\anaconda3\envs\nlp did not contain ['cudart64_110.dll', 'cudart64_120.dll'] as expected! Searching further paths...
  warn(msg)
c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('C:/Users/panta/anaconda3/envs/nlp/Library/mingw-w64/bin'), WindowsPath('C:/Users/panta/anaconda3/envs/nlp/Library/usr/bin')}
  warn(msg)


In [2]:
BASE_MODEL = "facebook/opt-350m"

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

tokenizer.pad_token_id = 0

In [3]:
model = prepare_model_for_int8_training(model)
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 1572864 || all params: 332769280 || trainable%: 0.472659014678278


In [5]:
from datasets import load_dataset

def generate_prompt(data_point):
    return f"### Caption: {data_point['caption_string']}\n### Tags:{data_point['tag_string']}"

data = load_dataset("json", data_files=os.path.join("dataset", "train_data.json"))
# data = data["train"].train_test_split(test_size=0.05, shuffle=True, seed=42)
data = data.map(lambda x: tokenizer(generate_prompt(x)))
data

Found cached dataset json (C:/Users/panta/.cache/huggingface/datasets/json/default-fe39fbd7ec44fd7c/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\panta\.cache\huggingface\datasets\json\default-fe39fbd7ec44fd7c\0.0.0\fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e\cache-f5023251e052bb14.arrow


DatasetDict({
    train: Dataset({
        features: ['caption_string', 'tag_string', 'input_ids', 'attention_mask'],
        num_rows: 19950
    })
})

In [6]:
trainer = transformers.Trainer(
    model=model, 
    train_dataset=data['train'],
    # eval_dataset=data['test'],

    args=transformers.TrainingArguments(
        per_device_train_batch_size=4, 
        gradient_accumulation_steps=32,
        num_train_epochs=3,
        learning_rate=3e-4, 
        fp16=True,
        logging_steps=2,
        optim="adamw_torch",
        output_dir='outputs',
        # evaluation_strategy="steps",
        # eval_steps=20,
        # save_strategy="steps",
        # save_steps=20,
        # save_total_limit=2,
        # report_to="tensorboard"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

  0%|          | 0/465 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\autograd\_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'loss': 3.4171, 'learning_rate': 0.0002987096774193548, 'epoch': 0.01}
{'loss': 3.227, 'learning_rate': 0.0002974193548387097, 'epoch': 0.03}
{'loss': 3.1362, 'learning_rate': 0.0002961290322580645, 'epoch': 0.04}
{'loss': 3.0572, 'learning_rate': 0.00029483870967741933, 'epoch': 0.05}
{'loss': 2.993, 'learning_rate': 0.00029354838709677415, 'epoch': 0.06}
{'loss': 2.9182, 'learning_rate': 0.00029225806451612903, 'epoch': 0.08}
{'loss': 2.9066, 'learning_rate': 0.00029096774193548386, 'epoch': 0.09}
{'loss': 2.8348, 'learning_rate': 0.0002896774193548387, 'epoch': 0.1}
{'loss': 2.7995, 'learning_rate': 0.0002883870967741935, 'epoch': 0.12}
{'loss': 2.7723, 'learning_rate': 0.0002870967741935484, 'epoch': 0.13}
{'loss': 2.7399, 'learning_rate': 0.0002858064516129032, 'epoch': 0.14}
{'loss': 2.7328, 'learning_rate': 0.00028451612903225803, 'epoch': 0.15}
{'loss': 2.7017, 'learning_rate': 0.00028322580645161286, 'epoch': 0.17}
{'loss': 2.6616, 'learning_rate': 0.00028193548387096774, 'ep

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
model.save_pretrained(os.path.join('loras','tagger-v2'))

In [9]:
batch = tokenizer("### Caption: Eromame's art features a female Corrin and Incineroar in armor from Fire Emblem, with long hair, a hairband, pointy ears, and an open mouth, against a simple background, and posted on their Twitter.", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=200)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\transformers\generation\utils.py:1250: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\transformers\generation\utils.py:1440: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\autograd\_functions.p



 ### Caption: Eromame's art features a female Corrin and Incineroar in armor from Fire Emblem, with long hair, a hairband, pointy ears, and an open mouth, against a simple background, and posted on their Twitter.
### Tags:1girl, armor, fire_emblem, long_hair, closed_mouth, closed_mouth, closed_mouth_open_mouth, closed_mouth_open_mouth, hat, hat_on_head, solo, solo_bow, pointy_ears, pointy_ears_on_head, pointy_ears_on_hair, solo_bow, solo_bow_on_head, solo_bow_on_head, solo_bow_on_legs, solo_bow_on_legs, solo_bow_on_legs_open_mouth, solo_bow_on_legs_closed, solo_bow_on_legs_open_mouth, solo_bow_on_legs_closed, solo_bow_on_legs_open_mouth, solo_bow_on_legs_closed, solo_bow_on_legs_closed_


In [ ]:
%load_ext tensorboard